In [6]:
import requests
import pandas as pd
from itertools import product

In [7]:
def fetch(year, areas):
    """
    Fetches data from the hvakosterstrommen API
    
    Parameters
    ----------
    year : int
    areas : list of str
        For example ["NO1", "NO2"]

    Returns
    -------
    pd.DataFrame
    
    Notes
    -----
    For Sweden, see https://www.elprisetjustnu.se/elpris-api
    """
    URL = "https://www.hvakosterstrommen.no/api/v1/prices/{}/{}-{}_{}.json"
    
    _data = []
    for month, day, area in product(range(1, 13), range(1, 32), areas):
        try:
            if pd.Timestamp(f"{year}-{month}-{day}") > pd.Timestamp.now():
                print(f"Cannot fetch data from the future. Returning data until "
                      f"{year}-{str(month).zfill(2)}-{str(day).zfill(2)}")
                break
        except ValueError:
            pass
        
        _response = requests.get(URL.format(year, str(month).zfill(2), str(day).zfill(2), area))

        if _response.status_code == 404:
            print(f"No data found for "
                  f"{year}-{str(month).zfill(2)}-{str(day).zfill(2)}") if day <= 28 else None
            continue

        _response = pd.DataFrame(_response.json()).drop(columns=["time_end", "NOK_per_kWh", "EXR"])
        _response.rename(columns={"EUR_per_kWh": f"{area} EUR_per_MWh"}, inplace=True)
        #change EUR_per_kWh to EUR_per_MWh
        _response[f"{area} EUR_per_MWh"] = _response[f"{area} EUR_per_MWh"] * 1000
        _response["time_start"] = pd.to_datetime(_response["time_start"], 
                                                 utc=True) + pd.Timedelta(hours=1)
        _response.set_index("time_start", inplace=True)

        _data.append(_response)
    return pd.concat(_data, axis=1).T.groupby(level=0).first().T

In [8]:
data = fetch(2023, ["NO1", "NO2", "NO3", "NO4", "NO5"])

In [9]:
data

,NO1 EUR_per_MWh,NO2 EUR_per_MWh,NO3 EUR_per_MWh,NO4 EUR_per_MWh,NO5 EUR_per_MWh
time_start,,,,,
2023-01-01 00:00:00+00:00,119.32,119.32,23.35,23.35,119.32
2023-01-01 01:00:00+00:00,108.83,108.83,23.28,23.28,108.83
2023-01-01 02:00:00+00:00,102.39,102.39,23.37,23.37,102.39
2023-01-01 03:00:00+00:00,92.36,92.36,23.67,23.67,92.36
2023-01-01 04:00:00+00:00,82.66,82.66,24.12,24.12,82.66
...,...,...,...,...,...
2023-12-31 19:00:00+00:00,62.88,62.88,39.64,38.14,62.88
2023-12-31 20:00:00+00:00,62.60,62.60,34.89,34.89,62.60
2023-12-31 21:00:00+00:00,62.25,62.25,29.60,29.60,62.25


In [10]:
data.to_pickle("./2023_spot_norway.pkl")